In [1]:
import pandas as pd
import re
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# use joined data
# df_joined = pd.read_csv ('Joined_data.csv')
df_joined = pd.read_csv ('../joined_data/joined_data_CHR.csv')

df_joined


geo = gpd.read_file("zip://../ct_data/tl_2019_55_tract.zip")
geo.plot()
print(geo.shape)
geo.head(1)

geo['GEOID']=geo['GEOID'].astype(np.int64)
joined = gpd.GeoDataFrame(pd.merge(df_filtered,geo,how="left",left_on="ct",right_on = "GEOID"))


,ct,male,female,Total_x,.50,.50-.99,1.00-1.24,1.25-1.49,1.50-1.84,1.85-1.99,...,60000-74999,75000-99999,100000-124999,125000-149999,150000-199999,200000,household_median_income,pop,no_hispanic,hispanic
0,55001950100,1504,1431,2935,46,139,117,42,151,207,...,116,196,39,50,33,67,55562.00,2935,2923,12
1,55001950201,676,667,1343,38,172,52,47,76,46,...,76,34,48,2,9,3,41071.00,1343,1319,24
2,55001950202,1414,1167,2578,118,268,164,148,191,72,...,167,88,29,12,13,16,39897.00,2581,2509,72
3,55001950400,2260,2269,4337,305,512,464,228,335,145,...,147,179,66,25,33,16,36909.75,4529,4331,198
4,55001950501,841,670,1511,25,238,34,170,118,41,...,42,55,27,11,25,10,32805.00,1511,1442,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,55141011300,2090,2142,4133,103,225,141,306,332,28,...,253,219,118,88,44,24,46504.40,4232,4207,25
1386,55141011400,2205,2958,5145,181,285,555,399,193,200,...,405,283,125,42,89,26,40607.80,5163,5097,66
1387,55141011500,2897,2936,5820,99,79,121,158,150,139,...,362,374,400,147,246,81,75201.50,5833,5793,40
1388,55141011600,2570,2472,5014,257,316,167,255,141,479,...,195,314,280,36,40,16,52031.00,5042,5042,0


In [3]:
import pandas as pd
import re
import geopandas as gpd
import numpy as np
import libpysal.weights as weights
import libpysal

In [4]:
kd = libpysal.cg.KDTree(np.array(df_joined[["INTPTLAT","INTPTLON"]].values))
wnn = libpysal.weights.KNN(kd, 3)
nearest_pt = pd.DataFrame().from_dict(wnn.neighbors, orient="index")
for i in range(nearest_pt.shape[1]):
    nearest_pt = nearest_pt.rename({i:f"n_pt_{i}"}, axis=1)
nearest_pt.head(1)
print(nearest_pt)



KeyError: "None of [Index(['INTPTLAT', 'INTPTLON'], dtype='object')] are in the [columns]"

In [ ]:
df_joined.reset_index(inplace=True)
df_joined

In [ ]:

df_joined = df_joined.join(nearest_pt)
df_joined.head(1)



In [ ]:
col_names_joined = pd.Series(df_joined.columns)
col_names_joined
print('All joined attributes \n',df_joined.columns.to_list(),'\n')
algo_input_addtri = []
# algo_input_addtri = ["n_pt_0", "n_pt_1", "n_pt_2"]
# algo_input_addtri.append("male")
# algo_input_addtri.append("female")
geo_attrib = ['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC',
       'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry', 'ct', 'index']
for each in col_names_joined:
    if((each not in algo_input_addtri) and(each not in geo_attrib) ):
        algo_input_addtri.append(each)
        print(each)
    

# print(algo_input_addtri)

In [ ]:
import copy
df_joined[algo_input_addtri].to_csv(r'synthetic_data_joined.txt', header=None, index=True, sep=',')
print(df_joined.shape)
final_input_attri = copy.copy(algo_input_addtri)
final_input_attri.insert(0,"id")
print(final_input_attri)

In [ ]:

synthetic_data_input = pd.read_table(r'synthetic_data_joined.txt', sep=',', names=final_input_attri)
synthetic_data_input = synthetic_data_input.set_index("id")
# synthetic_data_input

np.where(pd.isnull(synthetic_data_input))
# synthetic_data_input.loc[74].tolist()
# w_voronoi = weights.Voronoi.from_dataframe(prev)

In [ ]:
# prev[["DP03_0001E", "DP03_0001PE", 
#                 "DP03_0002E", 
#                 "DP03_0002PE", "DP03_0003E"]] = synthetic_data_input[["DP03_0001E", "DP03_0001PE", 
#                 "DP03_0002E", 
#                 "DP03_0002PE", "DP03_0003E"]]
# prev.head(1)

#find last 
colums_name = synthetic_data_input.columns
for i in range(len(synthetic_data_input.columns)):
#     print(colums_name[i])
    if("hispanic" == colums_name[i]):
        print("last index", i+1)
    if("n_pt_2" == colums_name[i]):
        print("last neighbor index", i+1)
synthetic_data_input.shape

In [ ]:
# w_voronoi = weights.Voronoi.from_dataframe(prev)

In [ ]:
# !rm -rf result_synthetic_data_joined.txt
!python STICC_main.py --fname=synthetic_data_joined.txt --oname=result_synthetic_data_joined.txt --attr_idx_start=1 \
--attr_idx_end=93 --spatial_idx_start=94 --spatial_idx_end=96 \
--spatial_radius=3 --number_of_clusters 3 --lambda_parameter 10e-2 --beta 5 --maxIters 20 